In [1]:
import numpy as np
import pandas as pd
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.metrics import mean_absolute_error 

Using TensorFlow backend.


In [2]:
train = pd.read_csv("./Data/train_2017.csv")
test = pd.read_csv("./Data/test_2017.csv")

print(train.shape)

train.head()

(1591497, 5)


,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,yearbuilt,taxvaluedollarcnt
0,3.0,6.0,3660.0,2003.0,655280.0
1,2.0,3.0,1152.0,1911.0,196726.0
2,2.0,2.0,1230.0,1977.0,218275.0
3,1.0,3.0,1464.0,1958.0,57901.0
4,2.0,2.0,1185.0,1949.0,270354.0


In [3]:
print(test.shape)
test.head()

(530499, 5)


,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,yearbuilt,taxvaluedollarcnt
0,2.0,3.0,1900.0,1985.0,293880.0
1,2.5,3.0,2236.0,1987.0,798691.0
2,2.0,3.0,1291.0,1956.0,419180.0
3,4.5,4.0,4425.0,2010.0,1687000.0
4,2.0,3.0,1620.0,1953.0,293830.0


In [4]:
X_train = train[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 
        'yearbuilt']].values
y_train = train['taxvaluedollarcnt'].values

X_test = test[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 
        'yearbuilt']].values
y_test = test['taxvaluedollarcnt'].values

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#X = scaler.fit_transform(X)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1591497, 4), (530499, 4), (1591497,), (530499,))

In [7]:

nnmodel = Sequential()
# Input => Hidden
nnmodel.add(Dense(32, input_dim=4, activation='relu'))
# Hidden
nnmodel.add(Dense(32, activation='relu'))
#nnmodel.add(Dense(32, activation='relu'))
#nnmodel.add(Dense(32, activation='relu'))
# Output Layer
nnmodel.add(Dense(1, activation='linear'))

# Compile
nnmodel.compile(loss='mean_absolute_error',
                    optimizer='adam',
                    metrics=['mean_absolute_error'])

nnmodel.summary()

W0925 11:58:45.993171 4478809536 deprecation.py:506] From /anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                160       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,249
Trainable params: 1,249
Non-trainable params: 0
_________________________________________________________________


In [8]:
nnmodel.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=.3, verbose=1)
scores = nnmodel.evaluate(X_test, y_test)
print(f'{nnmodel.metrics_names[1]}: {scores[1]*100}')

Train on 1114047 samples, validate on 477450 samples
Epoch 1/10
1114047/1114047 [==============================] - 27s 24us/sample - loss: 213000.8385 - mean_absolute_error: 213000.8438 - val_loss: 208528.2338 - val_mean_absolute_error: 208527.7969
Epoch 2/10
1114047/1114047 [==============================] - 28s 25us/sample - loss: 208502.9721 - mean_absolute_error: 208501.5625 - val_loss: 208269.6272 - val_mean_absolute_error: 208269.4688
Epoch 3/10
1114047/1114047 [==============================] - 28s 25us/sample - loss: 208412.3327 - mean_absolute_error: 208412.8281 - val_loss: 208760.6680 - val_mean_absolute_error: 208759.7812
Epoch 4/10
1114047/1114047 [==============================] - 28s 25us/sample - loss: 208356.6067 - mean_absolute_error: 208356.3594 - val_loss: 208001.8678 - val_mean_absolute_error: 208001.5625
Epoch 5/10
1114047/1114047 [==============================] - 27s 25us/sample - loss: 208316.9387 - mean_absolute_error: 208317.3906 - val_loss: 207958.5350 - val_

In [ ]:
# Function to create model
def create_model():
    # create model
    nnmodel = Sequential()
    # Input => Hidden
    nnmodel.add(Dense(32, input_dim=4, activation='relu'))
    # Hidden
    nnmodel.add(Dense(32, activation='relu'))
    nnmodel.add(Dense(32, activation='relu'))
    nnmodel.add(Dense(32, activation='relu'))
    # Output Layer
    nnmodel.add(Dense(1, activation='linear'))

    # Compile
    nnmodel.compile(loss='mean_absolute_error',
                    optimizer='adam',
                    metrics=['mean_absolute_error'])
    return model
  
model = KerasRegressor(build_fn=create_model, verbose=1)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10,20,40,60,80]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=5)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

In [6]:

def baseline_model(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(32, activation='relu', 
                    kernel_regularizer = 'l2', 
                    kernel_initializer = 'normal', 
                    input_shape=(4,)))
   # model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', 
                    kernel_regularizer = 'l2',
                    kernel_initializer = 'normal'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear',
                    kernel_regularizer = 'l2', 
                    kernel_initializer='normal'))
    model.compile(loss='mean_absolute_error', optimizer=optimizer, metrics=['mean_absolute_error'])
    return model

def gridSearch_neural_network(X_train, y_train):
    
    print("Train Data:", X_train.shape)
    print("Train label:", y_train.shape)
    # evaluate model with standardized dataset
    estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=10, batch_size=5, verbose=1)
    
    # grid search epochs, batch size and optimizer
    optimizers = ['rmsprop', 'adam']
    #dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    init = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 150]
    batches = [5, 10, 20]
    weight_constraint = [1, 2, 3, 4, 5]
    param_grid = dict(optimizer=optimizers, 
                      #dropout_rate=dropout_rate, 
                      epochs=epochs, 
                      batch_size=batches, 
                      #weight_constraint=weight_constraint, 
                      #init=init
                     )
    
    grid = GridSearchCV(estimator=estimator, param_grid=param_grid)
    grid_result = grid.fit(X_train, y_train)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
gridSearch_neural_network(X_train, y_train)

Train Data: (1591497, 4)
Train label: (1591497,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
import pickle

In [ ]:
pickle.dump(nnmodel, open('zillow_nn_model.pkl','wb'))